In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import rasterio
import glob
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

In [2]:
def read_tif_files(directory_path, pattern):
    """
    Read all TIFF files matching the pattern from directory
    pattern: 'NDMI' or 'NDVI'
    Returns: Dictionary with filenames as keys and numpy arrays as values
    """
    # Get list of files in directory
    try:
        files = os.listdir(directory_path)
        # Filter for TIFF files matching the pattern
        tif_files = sorted([f for f in files if f.startswith(pattern) and f.endswith('.tif')])
        
        if not tif_files:
            raise ValueError(f"No {pattern} TIFF files found in {directory_path}")
            
        data_dict = {}
        for filename in tif_files:
            file_path = os.path.join(directory_path, filename)
            with rasterio.open(file_path) as src:
                data = src.read(1)
                data_dict[filename] = data
                
        print(f"Loaded {len(data_dict)} {pattern} files from {directory_path}")
        return data_dict
        
    except Exception as e:
        print(f"Error reading {pattern} files from {directory_path}: {str(e)}")
        raise

In [25]:
def prepare_features(ndmi_dir, ndvi_dir, precip_df):
    """
    Prepare features from TIFF files and precipitation data
    precip_df: DataFrame with columns [tiff_file, precipitation_sum, irrigation]
    """
    # Read TIFF files
    print("Reading TIFF files...")
    ndmi_dict = read_tif_files(ndmi_dir, 'NDMI')
    ndvi_dict = read_tif_files(ndvi_dir, 'NDVI')
    
    # Function to extract dates from filename
    def extract_dates(filename):
        # Extract start date from NDMI_YYYY-MM-DD_to_YYYY-MM-DD.tif
        start_date = filename.split('_')[1]
        return start_date
    
    # Create mapping between NDMI and NDVI files using start dates
    ndvi_mapping = {}
    for ndvi_file in ndvi_dict.keys():
        ndvi_date = extract_dates(ndvi_file)
        ndvi_mapping[ndvi_date] = ndvi_file
    
    # Find extra NDVI file
    ndmi_dates = set(extract_dates(f) for f in ndmi_dict.keys())
    ndvi_dates = set(extract_dates(f) for f in ndvi_dict.keys())
    extra_date = ndvi_dates - ndmi_dates
    if extra_date:
        extra_file = [f for f in ndvi_dict.keys() if extract_dates(f) in extra_date][0]
        print(f"Excluding extra NDVI file: {extra_file}")
    
    # Sort precipitation data by date
    precip_df = precip_df.sort_values('tiff_file')
    
    features = []
    targets = []
    dates = []
    
    print("\nPreparing feature vectors...")

    # Iterate through precipitation data to ensure alignment
    for i in range(1, len(precip_df)):
        current_ndmi_file = precip_df.iloc[i]['tiff_file']
        previous_ndmi_file = precip_df.iloc[i-1]['tiff_file']

        
        # Get corresponding NDVI files
        current_date = extract_dates(current_ndmi_file)
        previous_date = extract_dates(previous_ndmi_file)
        
        current_ndvi_file = ndvi_mapping.get(current_date)
        previous_ndvi_file = ndvi_mapping.get(previous_date)

        # print(previous_ndmi_file)
        # print(previous_ndvi_file)
        
        # Check if we have all required data
        if (current_ndmi_file in ndmi_dict and previous_ndmi_file in ndmi_dict and
            current_ndvi_file and previous_ndvi_file):  # Check if NDVI files exist
            print('Exists')
        else:
            print('Does not exist')
            continue
            
    #         # Current values
    #         ndmi_current = ndmi_dict[current_ndmi_file].flatten()
    #         ndvi_current = ndvi_dict[current_ndvi_file].flatten()
    #         precip = precip_df.iloc[i]['precipitation_sum']
            
    #         # Previous values
    #         ndmi_prev = ndmi_dict[previous_ndmi_file].flatten()
    #         ndvi_prev = ndvi_dict[previous_ndvi_file].flatten()
            
    #         # Combine features
    #         feature_row = np.concatenate([
    #             ndmi_current,
    #             ndvi_current,
    #             ndmi_prev,
    #             ndvi_prev,
    #             [precip]
    #         ])
            
    #         features.append(feature_row)
    #         targets.append(precip_df.iloc[i]['irrigation'])
    #         dates.append(current_ndmi_file)
    
    # print(f"\nDataset summary:")
    # print(f"Total samples: {len(features)}")
    # print(f"Feature vector size: {features[0].shape[0]} elements")
    # print(f"Date range: {dates[0]} to {dates[-1]}")
    
    # return np.array(features), np.array(targets), dates

In [11]:

def train_irrigation_model(features, targets):
    # Time series split for validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    # XGBoost parameters
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.1,
        'max_depth': 6,
        'n_estimators': 100,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'gamma': 0
    }
    
    models = []
    scores = []
    
    for train_idx, val_idx in tscv.split(features):
        X_train, X_val = features[train_idx], features[val_idx]
        y_train, y_val = targets[train_idx], targets[val_idx]
        
        model = xgb.XGBRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=10,
            verbose=False
        )
        
        pred = model.predict(X_val)
        score = r2_score(y_val, pred)
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        
        models.append(model)
        scores.append({'R2': score, 'RMSE': rmse})
    
    best_model_idx = np.argmax([s['R2'] for s in scores])
    return models[best_model_idx], scores

In [12]:
def optimize_irrigation(model, features, current_irrigation):
    optimal_irrigation = model.predict(features.reshape(1, -1))[0]
    
    # Apply constraints
    min_irrigation = 0
    max_irrigation = 50  # Adjust based on your requirements
    
    optimal_irrigation = np.clip(optimal_irrigation, min_irrigation, max_irrigation)
    savings = current_irrigation - optimal_irrigation if optimal_irrigation < current_irrigation else 0
    
    return optimal_irrigation, savings

In [13]:
ndmi_dir = "data/NDMI_Weekly_Exports/NDMI_Weekly_Exports_2019_2024"
ndvi_dir = "data/NDVI_Weekly_Exports/NDVI_Weekly_Exports_2019_2024"
# Load precipitation and irrigation data
precip_df = pd.read_csv('data/precipitation_sums.csv')  # Should contain tiff_file, precipitation_sum, irrigation

In [26]:
# Prepare features
print("Preparing features...")
# features, targets, dates = prepare_features(ndmi_dir, ndvi_dir, precip_df)
prepare_features(ndmi_dir, ndvi_dir, precip_df)

Preparing features...
Reading TIFF files...
Loaded 199 NDMI files from data/NDMI_Weekly_Exports/NDMI_Weekly_Exports_2019_2024
Loaded 200 NDVI files from data/NDVI_Weekly_Exports/NDVI_Weekly_Exports_2019_2024
Excluding extra NDVI file: NDVI_2019-07-02_to_2019-07-09.tif

Preparing feature vectors...
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Exists
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not exist
Does not ex